*Analytical Information Systems*

# Worksheet 5 - Big Data and Streaming

Matthias Griebel<br>
Lehrstuhl für Wirtschaftsinformatik und Informationsmanagement

SS 2020

## Exercises

### 1. MapReduce

__Sales analysis__

You need to run a company-wide sales analysis. Your company uses a MapReduce system to handle the massive transaction data.

We will have a look at the data first:


In [14]:
library(tidyverse)
sales <- read_csv('https://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/data/sales.csv')
sales %>% head(10)

Parsed with column specification:
cols(
  date = col_date(format = ""),
  customerID = col_double(),
  productID = col_double(),
  payment = col_character(),
  amount = col_double(),
  price = col_double(),
  cost = col_double(),
  category = col_character()
)



date,customerID,productID,payment,amount,price,cost,category
<date>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
2017-01-16,64292,8403,paypal,3,560.74,234.89,emergency
2017-08-16,41174,7234,paypal,3,351.14,171.11,specialty
2017-10-26,49737,32738,paypal,3,343.38,105.14,emergency
2017-11-24,24021,70159,cash,2,905.96,345.40,emergency
2017-02-13,78762,2002,cash,2,799.99,407.30,emergency
2017-07-18,79148,86205,credit card,1,284.07,132.35,emergency
2017-08-23,79148,40784,cash,3,125.79,47.53,specialty
2017-11-06,23090,16224,paypal,3,85.77,36.61,specialty
2017-09-28,12307,82560,credit card,2,658.88,330.44,emergency


Define the corresponding Map and Reduce functions:

__Map__: Calculates the total profit for each product id within each subset

In [0]:
calculate_profit <- function(df){
    df %>%
        mutate(profit=(price-cost)*amount) %>% 
        group_by(productID) %>%
        summarise(total_profit = sum(profit))
}

__Reduce__: Adds up the profit for each different product id

In [0]:
reduce_profit <- function(df){
    df %>%
        summarise(productID=productID[1],
                  total_profit=sum(total_profit)) 
}

In [17]:
sales %>% #Input
    split(sample(rep(1:5, 1000))) %>% #Splitting
    map(calculate_profit) %>% #Mapping
    map_df(rbind) %>% group_split(productID)%>% #Shuffling
    map(reduce_profit) %>% #Reduce
    map_df(cbind) %>% arrange(desc(total_profit)) %>%  #Merge and Sort
    head(10) #Display only top 10

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)



,productID,total_profit
,<dbl>,<dbl>
1,43446,171232.25
2,24221,111657.54
3,60974,88118.82
4,70159,84644.56
5,89266,73408.04
6,86064,68414.58
7,9947,64715.40
8,61070,64571.40
9,62077,63238.50


### 2. Stream Processing

1. Why do stateful operation complicate parallelization?

In [0]:
 # Need to join results across parallel computations

2. Why do parallelization operations complicate fault-tolerance?

In [0]:
 # Handeling the failure of a node is more complicated; e.g. if we need to ensure exactly-once semantics